## Transfer Learning with Keras (ResNet101V2) for Object Classification 

In [ ]:
# Import all the required libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pathlib
import os
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pathlib

In [ ]:
# Parameters
img_height, img_width, img_channel = 224, 224, 3
batch_size = 32
n_epochs = 10
initial_learning_rate = 0.001
classes = 5

# mean and std normalization values for ImageNet from https://discuss.pytorch.org/t/discussion-why-normalise-according-to-imagenet-mean-and-std-dev-for-transfer-learning/115670
MEAN = 255 * np.array([0.485, 0.456, 0.406]) 
STD = 255 * np.array([0.229, 0.224, 0.225]) 

# Import dataset, this is an example dataset containing 5 different classe of flower photos
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
# data_dir = pathlib.Path(data_dir) # we can devide the images into training and test folders
data_dir = pathlib.Path('data/dataset_train/') # 80% training, 10% validation
test_dir = pathlib.Path('data/dataset_test/') # 10% testing
validation_split = 0.1
n_seed = 123

In [ ]:
# Split the Data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split = validation_split,
  subset="training",
  seed = n_seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)
  
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split = validation_split,
  subset="validation",
  seed = n_seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names   # classes 
print(class_names)

In [ ]:
# Normalizing data 
def process(image,label):
    image = tf.cast((image - MEAN) / STD ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
val_ds = val_ds.map(process)

In [ ]:
# Visualize our data
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(25):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# Import ResNet101V2 Pre-trained Model
# you can choose other pretrained models from here https://keras.io/api/applications/
model = Sequential()

pretrained_model= tf.keras.applications.ResNet101V2(include_top=False,
                   input_shape = (img_height, img_width, img_channel), 
                   pooling = 'avg', classes = classes,
                   weights = 'imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

model.add(pretrained_model)

# Adding a fully connected and output layer where actual learning can take place
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

In [ ]:
# Exponential decay schedule: learning rate between epochs or iterations as the training progresses
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# Model compiler with respect to Learning Rate Scheduler
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Model compiler without Exponential decay schedule
model.compile(optimizer = Adam(learning_rate = initial_learning_rate),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Model Training (model fit)
# ReduceLROnPlateau: Reduce learning rate  
tf.keras.callbacks.ReduceLROnPlateau(
    monitor="accuracy",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
history = model.fit(train_ds, validation_data=val_ds, callbacks=[reduce_lr])

In [ ]:
# Model Training (model fit)
# Weighted loss class imbalance
class_weight = {0: 633, 
                1: 898, 
                2: 641, 
                3: 699, 
                4: 799}

history = model.fit(train_ds, validation_data=val_ds, epochs= n_epochs, class_weight=class_weight)

In [ ]:
# Model Evaluation
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# Model Inference
predicted_class_indices = [] # initializing an array to save prediction values
GT_class_indices = [] # initializing an array to keep the ground trurth values

for folder in os.listdir(test_dir):
    for img in os.listdir(str(test_dir) + '/' + str(folder)):
        image = cv2.imread(str(test_dir) + '/' + str(folder) + '/' + img)
        # Normalizing data
        # Note: next two lines should be uncommented in case of skipping normalzing the training data
        image = np.array(image)
        image = (image - MEAN) / STD
        
        image_resized = cv2.resize(image, (img_height,img_width)) # resize the image to the learnt size in the training
        image = np.expand_dims(image_resized,axis=0) # expanding the image dimesion to 4D  
        
        # Prediction on testing data
        pred = model.predict(image)
        
        output_class = class_names[np.argmax(pred)] 
        predicted_class_indices.append(output_class) # prediction array
        GT_class_indices.append(folder) # ground truth array
        
        print("The GT is", str(folder)," and the predicted class is", output_class)

In [ ]:
# Confusion Matrix for evaluation of the model
cm = confusion_matrix(predicted_class_indices, GT_class_indices)
cm_df = pd.DataFrame(cm)

# Plotting the confusion matrix
plt.figure(figsize = (5,4))
sns.heatmap(cm_df, annot = True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
# Total recall and precision for evaluation of the model
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
recall = np.mean(recall)
precision = np.mean(precision)
print(recall, precision)

In [ ]:
# Saving prediction vs ground truth in a csv file
results=pd.DataFrame({"Filename":GT_class_indices,
                      "Predictions":predicted_class_indices})
results.to_csv(str(test_dir) + "results.csv",index=False)